In [1]:
import numpy as np
import pandas as pd


In [2]:
# =========================
# ИСХОДНЫЕ ДАННЫЕ
# =========================

# Исторические данные по выполненным задачам
data = [
    (2.0, 40), (1.5, 40), (1.0, 40), (3.0, 40), (1.5, 40), (2.0, 40), (1.5, 40),
    (2.0, 41), (1.5, 41), (2.0, 41), (1.0, 41), (1.5, 41), (1.0, 41), (2.0, 41),
    (1.0, 41), (1.5, 41), (2.0, 41), (1.5, 41),
    (2.0, 42), (2.5, 42), (1.5, 42), (2.0, 42)
]

df = pd.DataFrame(data, columns=["hours", "week"])


In [3]:
# =========================
# ДИСКРЕТНЫЕ РАСПРЕДЕЛЕНИЯ
# =========================

weekly_stats = df.groupby("week").agg(
    tasks_done=("hours", "count"),
    hours_done=("hours", "sum")
)

tasks_per_week = weekly_stats["tasks_done"].to_numpy()
hours_per_week = weekly_stats["hours_done"].to_numpy()

tasks_per_week, hours_per_week


(array([ 7, 11,  4]), array([12.5, 17. ,  8. ]))

In [4]:
# =========================
# ПАРАМЕТРЫ ФАЗЫ 5
# =========================

N_TASKS_PHASE5 = 5
N_ITER = 20000
rng = np.random.default_rng(42)


In [5]:
def simulate_by_task_count(n_tasks, tasks_per_week, rng):
    remaining = n_tasks
    weeks = 0
    while remaining > 0:
        weeks += 1
        remaining -= rng.choice(tasks_per_week)
    return weeks


In [6]:
weeks_model_1 = np.array([
    simulate_by_task_count(N_TASKS_PHASE5, tasks_per_week, rng)
    for _ in range(N_ITER)
])

weeks_model_1.mean(), np.quantile(weeks_model_1, 0.8, method="higher")


(np.float64(1.3308), np.int64(2))

In [7]:
# Трёхточечные оценки (O, BG, P)
triangular_params = [
    (1.0, 1.5, 2.5),
    (0.5, 0.75, 2.0),
    (0.5, 0.75, 1.5),
    (2.0, 2.5, 3.5),
    (2.5, 3.5, 5.0),
]


In [8]:
def sample_total_task_duration(params, rng):
    return sum(rng.triangular(o, bg, p) for o, bg, p in params)


In [9]:
def simulate_by_hours(total_hours, hours_per_week, rng):
    remaining = total_hours
    weeks = 0
    while remaining > 0:
        weeks += 1
        remaining -= rng.choice(hours_per_week)
    return weeks


In [10]:
weeks_model_2 = []

for _ in range(N_ITER):
    total_duration = sample_total_task_duration(triangular_params, rng)
    weeks = simulate_by_hours(total_duration, hours_per_week, rng)
    weeks_model_2.append(weeks)

weeks_model_2 = np.array(weeks_model_2)

weeks_model_2.mean(), np.quantile(weeks_model_2, 0.8, method="higher")


(np.float64(1.33445), np.int64(2))

In [11]:
print("Модель 1 (по задачам):")
print("Средний срок:", weeks_model_1.mean())
print("80% срок:", np.quantile(weeks_model_1, 0.8, method='higher'))

print("\nМодель 2 (по трудозатратам):")
print("Средний срок:", weeks_model_2.mean())
print("80% срок:", np.quantile(weeks_model_2, 0.8, method='higher'))


Модель 1 (по задачам):
Средний срок: 1.3308
80% срок: 2

Модель 2 (по трудозатратам):
Средний срок: 1.33445
80% срок: 2


В первой модели случайной величиной является количество задач, выполняемых за неделю. Все задачи считаются равнозначными, вне зависимости от их реальной трудоёмкости. Такая модель опирается на историческую пропускную способность команды в терминах «задач в неделю» и не различает крупные и мелкие работы.

Во второй модели случайными величинами являются:

суммарные человеко-часы, которые команда может выделить за неделю;

длительности задач, сэмплируемые из треугольного распределения.

Таким образом, во второй модели учитывается фактический объём работ, а не только их количество.
